In [10]:
# import libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay
from sklearn.inspection import PartialDependenceDisplay

In [12]:
#read files we created in spatial join notebook
#the fully joined GDF to be able to make the 2020 data
parcelGdf = gpd.read_file('data/parcel_scag_geo.gpkg', driver = 'gpkg')
#the 2010 df for training
parcelcensus10 = gpd.read_file('parcelcensus10.gpkg', driver = 'gpkg')

DriverError: 'data/parcelcensus10.GPKG' not recognized as a supported file format.

### Apply Model to 2020 Dataset

Now we create a dataset to represent the conditions in the Inland Empire in 2020

In [4]:
census19 = gpd.read_file('data/census_2019.gpkg')
#memory dtypes
for col in census19.columns:
    if census19[col].dtype == 'float64': # skip the string column (srprec)
        census19[col] = census19[col].astype('float32')

In [ ]:
# Create a dataset for data as of 2020
parceldata_20=parcelGdf.copy()
parceldata_20.drop(columns=['warehouse_2010'],inplace=True)
parceldata_20.rename(columns={'warehouse_2020':'warehouse_start','built_2010s':'buit_within_decade'},inplace=True)

#join census 2019 dataset only
parceldata_20 = parceldata_20.to_crs(2229)
census19 = census19.to_crs(2229)
parcelcensus20 = gpd.sjoin(parceldata_20, census19, how='left', predicate='intersects')

# need to set APN to be index so that we can join this back to other information later on
parcelcensus20.set_index('APN',inplace=True)

In [ ]:
# define variables 
cols=parcelcensus20.columns.to_list()
xvars=[col for col in cols if col not in ('APN','building_class','year','acres','sqft','num_warehouses','buit_within_decade',
                                          'geometry','PID19','APN19','CITY','COUNTY','LU19','LU16','JURISDICTI','LU19_CLASS','SCAG_ZN_CO',
                                          'NAME','Shape_Leng','Shape_Area')]
yvar = 'buit_within_decade'

# create a dataframe with no NaNs
parceldata_20_model = parcelcensus20[xvars+[yvar]].dropna()

We now use the same RF model from before to predict the percent chance that a warehouse will be built within a decade on the 2020 data

In [ ]:
# apply predictions from previous model to test dataset
X_test=parceldata_20_model[xvars]
y_pred = rf.predict(X_test)

In [ ]:
# stop if lengths don't match
assert len(X_test)==len(y_pred)
# how many does it predict?
print('Predicted fraction True: {:.4f}'.format(y_pred.mean()))
print('Predicted number True: {:.0f}'.format(y_pred.sum()))

We will now create a dataframe of predictions to be able to visualize the results

In [ ]:
# create a dataframe of predictions about whether or not a parcel will have a warehouse built on it within 10 years & join this to the full df
predictions = pd.DataFrame(rf.predict_proba(X_test),columns = ['pred_noWH', 'pred_WH'])
predictions= predictions.join((X_test).reset_index())
predictions.set_index('APN',inplace=True)
predictions.head()

In [ ]:
# not a whole lot actally predicted to become warehouses... maybe visualize percentages above a certain point if census data doesn't fix
print(len(predictions[predictions.pred_WH>0.5]))
print(len(predictions[predictions.pred_WH>0.25]))